# Workout Data
This notebook is to prepare data for workout part on the webpage.

In [21]:
# Necessary packages
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sqlite3 import Error
from sqlalchemy import create_engine, text, MetaData, Table, UniqueConstraint
import os
from dotenv import load_dotenv
from datetime import datetime
from itertools import product
import pickle

## Load PostgreSQL database

In [22]:
# Load the current environment variables
load_dotenv()
# Get credentials to connect
database_url = os.getenv('DATABASE_URL')
# Create the engine to connect to the PostgreSQL cloud database
engine = create_engine(database_url)

In [23]:
# Verifing connection by checking the weight data from the database
pd.read_sql_query("SELECT * FROM weight_records", engine)

,date,weight
0,2018-04-26,83.00
1,2018-04-27,83.00
2,2018-04-28,83.00
3,2018-04-29,83.00
4,2018-04-30,83.00
...,...,...
2112,2024-02-06,85.50
2113,2024-02-07,84.30
2114,2024-02-08,84.30
2115,2024-02-09,84.85


## Clean up workout data

In [6]:
# Load raw file
df = pd.read_csv('workout.csv')
df.head()

,date,exercise_name,order,weight,rep
0,9/29/2020 7:10,Bent Over Row (Barbell),1,44.092452,5
1,9/29/2020 7:10,Bent Over Row (Barbell),2,66.138679,5
2,9/29/2020 7:10,Bent Over Row (Barbell),3,110.231131,5
3,9/29/2020 7:10,Bent Over Row (Barbell),4,154.323584,3
4,9/29/2020 7:10,Bent Over Row (Barbell),5,187.392923,8


In [7]:
# Manage date column
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.date
df.head()

,date,exercise_name,order,weight,rep
0,2020-09-29,Bent Over Row (Barbell),1,44.092452,5
1,2020-09-29,Bent Over Row (Barbell),2,66.138679,5
2,2020-09-29,Bent Over Row (Barbell),3,110.231131,5
3,2020-09-29,Bent Over Row (Barbell),4,154.323584,3
4,2020-09-29,Bent Over Row (Barbell),5,187.392923,8


In [10]:
# I am only interested and mainly tracking these 2 compound exercise.
df_bench = df[df['exercise_name']=='Bench Press (Barbell)']
df_chinup = df[df['exercise_name']=='Chin Up']

In [11]:
df_bench.head()

,date,exercise_name,order,weight,rep
19,2020-09-30,Bench Press (Barbell),1,214.950706,5
20,2020-09-30,Bench Press (Barbell),2,214.950706,4
21,2020-09-30,Bench Press (Barbell),3,214.950706,4
22,2020-09-30,Bench Press (Barbell),4,214.950706,4
59,2020-10-02,Bench Press (Barbell),1,44.092452,5


In [12]:
df_chinup.head()

,date,exercise_name,order,weight,rep
8,2020-09-29,Chin Up,1,0.0,10
9,2020-09-29,Chin Up,2,0.0,10
10,2020-09-29,Chin Up,3,0.0,9
45,2020-10-01,Chin Up,1,0.0,10
46,2020-10-01,Chin Up,2,0.0,10


In [13]:
# Function to drop warm-up sets
def filter_warmups(group):
    # If there are 4 or fewer sets, return the group as is
    if len(group) <= 4:
        return group
    
    # If there are more than 4 sets, apply the filter
    max_weight = group['weight'].max()
    warmup_threshold = max_weight * 0.7
    
    # Filtering out warm-up sets based on conditions
    conditions = ((group['order'] <= 4) & ((group['weight'] < warmup_threshold) | (group['rep'] < 5)))
    
    return group[~conditions]

In [14]:
# Group by date and apply the filter function
filtered_df = df_bench.groupby('date').apply(filter_warmups).reset_index(drop=True)
filtered_df

/var/folders/r4/2nk84h69255dy1q5js59_1b40000gn/T/ipykernel_52045/3910575545.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_df = df_bench.groupby('date').apply(filter_warmups).reset_index(drop=True)


,date,exercise_name,order,weight,rep
0,2020-09-30,Bench Press (Barbell),1,214.950706,5
1,2020-09-30,Bench Press (Barbell),2,214.950706,4
2,2020-09-30,Bench Press (Barbell),3,214.950706,4
3,2020-09-30,Bench Press (Barbell),4,214.950706,4
4,2020-10-02,Bench Press (Barbell),4,214.950706,5
...,...,...,...,...,...
1106,2024-01-28,Bench Press (Barbell),7,205.000000,5
1107,2024-02-01,Bench Press (Barbell),4,235.000000,5
1108,2024-02-01,Bench Press (Barbell),5,225.000000,5
1109,2024-02-01,Bench Press (Barbell),6,205.000000,6


In [16]:
# Create the table and insert both dataframes to the database
filtered_df.to_sql('bench_press', engine, if_exists='replace', index=False)
df_chinup.to_sql('chin_up', engine, if_exists='replace', index=False)

748

In [17]:
# Verifing data insertion, bench press
pd.read_sql_query("SELECT * FROM bench_press", engine)

,date,exercise_name,order,weight,rep
0,2020-09-30,Bench Press (Barbell),1,214.950706,5
1,2020-09-30,Bench Press (Barbell),2,214.950706,4
2,2020-09-30,Bench Press (Barbell),3,214.950706,4
3,2020-09-30,Bench Press (Barbell),4,214.950706,4
4,2020-10-02,Bench Press (Barbell),4,214.950706,5
...,...,...,...,...,...
1106,2024-01-28,Bench Press (Barbell),7,205.000000,5
1107,2024-02-01,Bench Press (Barbell),4,235.000000,5
1108,2024-02-01,Bench Press (Barbell),5,225.000000,5
1109,2024-02-01,Bench Press (Barbell),6,205.000000,6


In [24]:
# Verifing data insertion, chin up
pd.read_sql_query("SELECT * FROM chin_up", engine)

,date,exercise_name,order,weight,rep
0,2020-09-29,Chin Up,1,0.0,10
1,2020-09-29,Chin Up,2,0.0,10
2,2020-09-29,Chin Up,3,0.0,9
3,2020-10-01,Chin Up,1,0.0,10
4,2020-10-01,Chin Up,2,0.0,10
...,...,...,...,...,...
743,2024-01-27,Chin Up,4,0.0,6
744,2024-01-31,Chin Up,1,0.0,17
745,2024-01-31,Chin Up,2,0.0,11
746,2024-01-31,Chin Up,3,0.0,9


## Try plots

### First try weekly average load for Chin up

In [92]:
query_load ='''
            SELECT      chin_up.date, SUM((chin_up.weight+weight_records.weight*2/0.9) * chin_up.rep) AS load
            FROM        chin_up
            INNER JOIN  weight_records
            ON          chin_up.date = weight_records.date
            GROUP BY    chin_up.date
            '''

In [93]:
df = pd.read_sql_query(query_load, engine)
df

,date,load
0,2023-11-02,6186.000197
1,2022-03-30,8672.888591
2,2021-03-06,7452.889167
3,2020-10-31,6291.943292
4,2023-05-30,4551.999918
...,...,...
197,2023-07-18,6253.666856
198,2021-09-10,4499.999894
199,2022-07-23,5923.555772
200,2022-06-25,5738.444655


In [94]:
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df_average = df['load'].resample('W').mean().reset_index()

In [95]:
df_average['load'] = df_average['load'].interpolate()

In [96]:
px.line(df_average, x='date', y='load')

### Then Weekly Average peak one rep max for chin up

In [58]:
query_1rm = '''
            WITH a AS
            (
            SELECT      chin_up.date, chin_up.weight+weight_records.weight*2/0.9 AS weight, chin_up.rep
            FROM        chin_up
            INNER JOIN  weight_records
            ON          chin_up.date = weight_records.date
            ),
            b AS
            (
            SELECT      date, MAX(weight) AS weight
            FROM        a
            GROUP BY    date
            )
            SELECT      a.date, a.weight, MAX(a.rep) AS rep
            FROM        a
            INNER JOIN  b
            ON          a.date = b.date AND a.weight=b.weight
            GROUP BY    a.date, a.weight
            ORDER BY    a.date
            '''

In [59]:
df_1rm = pd.read_sql_query(query_1rm, engine)
df_1rm

,date,weight,rep
0,2020-09-29,189.239312,10
1,2020-10-01,189.957260,10
2,2020-10-03,190.675208,10
3,2020-10-10,207.268445,5
4,2020-10-13,186.666667,10
...,...,...,...
197,2024-01-18,189.333487,16
198,2024-01-21,189.555244,16
199,2024-01-24,189.777940,15
200,2024-01-27,189.555556,17


In [60]:
# Prepare a dictionary to calculate 1rm
one_rm_dict = {'1.0': 1, '2.0': 0.95, '3.0': 0.93, '4.0': 0.9, '5.0': 0.87, '6.0': 0.85,
                '7.0': 0.83, '8.0': 0.80, '9.0': 0.77, '10.0': 0.75, '11.0': 0.70, '12.0': 0.67,
                '13.0': 0.65, '14.0': 0.63, '15.0': 0.62, '16.0': 0.55, '17.0': 0.52, '18.0': 0.49,
                '19.0': 0.46, '20.0': 0.43}

In [64]:
# Convert the float to a string with one decimal place
df_1rm['rep'] = df_1rm['rep'].apply(lambda x: f"{x:.1f}")

# Get the corresponding value from the dictionary
df_1rm['percentage'] = df_1rm['rep'].map(one_rm_dict)

df_1rm

,date,weight,rep,percentage
0,2020-09-29,189.239312,10.0,0.75
1,2020-10-01,189.957260,10.0,0.75
2,2020-10-03,190.675208,10.0,0.75
3,2020-10-10,207.268445,5.0,0.87
4,2020-10-13,186.666667,10.0,0.75
...,...,...,...,...
197,2024-01-18,189.333487,16.0,0.55
198,2024-01-21,189.555244,16.0,0.55
199,2024-01-24,189.777940,15.0,0.62
200,2024-01-27,189.555556,17.0,0.52


In [65]:
# get 1 rep max
df_1rm['one_rep_max'] = df_1rm['weight']/df_1rm['percentage']
df_1rm

,date,weight,rep,percentage,one_rep_max
0,2020-09-29,189.239312,10.0,0.75,252.319082
1,2020-10-01,189.957260,10.0,0.75,253.276346
2,2020-10-03,190.675208,10.0,0.75,254.233611
3,2020-10-10,207.268445,5.0,0.87,238.239592
4,2020-10-13,186.666667,10.0,0.75,248.888889
...,...,...,...,...,...
197,2024-01-18,189.333487,16.0,0.55,344.242703
198,2024-01-21,189.555244,16.0,0.55,344.645899
199,2024-01-24,189.777940,15.0,0.62,306.093452
200,2024-01-27,189.555556,17.0,0.52,364.529915


In [66]:
df_1rm['date'] = pd.to_datetime(df_1rm['date'])
df_1rm.set_index('date', inplace=True)
df_1rm = df_1rm['one_rep_max'].resample('W').mean().reset_index()
df_1rm['one_rep_max'] = df_1rm['one_rep_max'].interpolate()

In [70]:
fig = px.line(df_1rm, x='date', y='one_rep_max')
fig.update_layout(
    xaxis=dict(
        type="date",
        rangeselector=dict(
            buttons=list([
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(count=2, label="1y", step="year", stepmode="backward"),
                dict(count=3, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        )
    )
)
fig.show()

### First try weekly average load for Bench Press

In [71]:
query_load ='''
            SELECT      date, SUM(weight*rep) AS load
            FROM        bench_press
            GROUP BY    date
            ORDER BY    date
            '''

In [72]:
df = pd.read_sql_query(query_load, engine)
df

,date,load
0,2020-09-30,3654.161995
1,2020-10-02,3869.112701
2,2020-10-05,4960.400898
3,2020-10-07,2810.893844
4,2020-10-09,2810.893844
...,...,...
236,2024-01-17,3645.000000
237,2024-01-20,4075.000000
238,2024-01-25,4525.000000
239,2024-01-28,3880.000000


In [73]:
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df = df['load'].resample('W').mean().reset_index()
df['load'] = df['load'].interpolate()

In [74]:
px.line(df, x='date', y='load')

### Then Weekly Average peak one rep max for Bench Press

In [84]:
query_1rm = '''
            WITH a AS
            (
            SELECT      date, MAX(weight) AS weight
            FROM        bench_press
            GROUP BY    date
            )
            SELECT      bench_press.date, bench_press.weight, MAX(bench_press.rep) AS rep
            FROM        a
            INNER JOIN  bench_press
            ON          a.date = bench_press.date AND a.weight = bench_press.weight
            GROUP BY    bench_press.date, bench_press.weight
            ORDER BY    bench_press.date
            '''

In [85]:
df_1rm = pd.read_sql_query(query_1rm, engine)
df_1rm

,date,weight,rep
0,2020-09-30,214.950706,5
1,2020-10-02,214.950706,5
2,2020-10-05,165.346697,10
3,2020-10-07,187.392923,5
4,2020-10-09,187.392923,5
...,...,...,...
236,2024-01-17,225.000000,5
237,2024-01-20,225.000000,5
238,2024-01-25,225.000000,6
239,2024-01-28,230.000000,6


In [86]:
# Convert the float to a string with one decimal place
df_1rm['rep'] = df_1rm['rep'].apply(lambda x: f"{x:.1f}")

# Get the corresponding value from the dictionary
df_1rm['percentage'] = df_1rm['rep'].map(one_rm_dict)

# get 1 rep max
df_1rm['one_rep_max'] = df_1rm['weight']/df_1rm['percentage']

df_1rm

,date,weight,rep,percentage,one_rep_max
0,2020-09-30,214.950706,5.0,0.87,247.069777
1,2020-10-02,214.950706,5.0,0.87,247.069777
2,2020-10-05,165.346697,10.0,0.75,220.462262
3,2020-10-07,187.392923,5.0,0.87,215.394164
4,2020-10-09,187.392923,5.0,0.87,215.394164
...,...,...,...,...,...
236,2024-01-17,225.000000,5.0,0.87,258.620690
237,2024-01-20,225.000000,5.0,0.87,258.620690
238,2024-01-25,225.000000,6.0,0.85,264.705882
239,2024-01-28,230.000000,6.0,0.85,270.588235


In [87]:
fig = px.line(df_1rm, x='date', y='one_rep_max')

In [88]:
fig.show()

In [83]:
# This part is to update data when I found an typo in the database recording
with engine.connect() as conn: 
    query = text("UPDATE bench_press SET weight = :weight WHERE date = :date AND \"order\" = :order")
    conn.execute(query, {'date': '2023-08-16', 'weight': 135, 'order': 2})
    conn.commit() 